<a href="https://colab.research.google.com/github/vialvaa/hello-world/blob/master/Algoritmos%20de%20clasificaci%C3%B3n%20y%20estudio%20estad%C3%ADstico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Algoritmos de clasificación**

En este notebook probaremos distintos algoritmos de aprendizaje supervisado para tratar de predecir qué actividad realizaba cada paciente de nuestro dataset.

Para comenzar, como en notebooks anteriores, cargaremos las librerías pertinentes y descargaremos los datos.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!wget https://www.dropbox.com/s/arwjxnjxx3mnert/test.csv -O test.csv

--2022-04-14 16:44:13--  https://www.dropbox.com/s/arwjxnjxx3mnert/test.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/arwjxnjxx3mnert/test.csv [following]
--2022-04-14 16:44:13--  https://www.dropbox.com/s/raw/arwjxnjxx3mnert/test.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc886246e92cb9f5bda1017da9ef.dl.dropboxusercontent.com/cd/0/inline/BjZ4Z9LPZV5kNMvSygxFvpSWCtM5R-L5DXqrPOQIkLJ__m65Z0Ye5g3nd-9xGMjT2b_Ggk_5RKG9jUUx5Kz04sS8IybTzItBLHJiBzLnCCOj-Ynkr5e1etPfU7oS7gUjmpuf2btIm2h9LiYdXnn3iYKiGsmdG2_89Jo766tUsEYjVw/file# [following]
--2022-04-14 16:44:14--  https://uc886246e92cb9f5bda1017da9ef.dl.dropboxusercontent.com/cd/0/inline/BjZ4Z9LPZV5kNMvSygxFvpSWCtM5R-L5DXqrPOQIkLJ__m65Z0Ye5g3nd-9xGMjT2b_Ggk_5RKG9jUUx

In [ ]:
!wget https://www.dropbox.com/s/4ps54iyfqkknw58/train.csv -O train.csv

--2022-04-14 16:44:15--  https://www.dropbox.com/s/4ps54iyfqkknw58/train.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/4ps54iyfqkknw58/train.csv [following]
--2022-04-14 16:44:15--  https://www.dropbox.com/s/raw/4ps54iyfqkknw58/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc6a42dcc681485aefe46f35ab13.dl.dropboxusercontent.com/cd/0/inline/BjZI_sFBjiY0mdFKHK_EcXTeMA5gnUWNfWJMH3uxnlgZj1Cx6u7HiG_KhxOmbgf1fqqJnNXJF91kAzVTOAWcxv-KrHMD1EkyHT_NNMwCQ80k7ss188yPbLJkHDslX7_dZzI9x_fV9q4H23ec2-BAs60uYX4dPaufsCvZvoW_HY7SoQ/file# [following]
--2022-04-14 16:44:16--  https://uc6a42dcc681485aefe46f35ab13.dl.dropboxusercontent.com/cd/0/inline/BjZI_sFBjiY0mdFKHK_EcXTeMA5gnUWNfWJMH3uxnlgZj1Cx6u7HiG_KhxOmbgf1fqqJnNXJF91kA

In [ ]:
df_train = pd.read_csv('train.csv')
X=df_train.values[:,:-1]
Y=df_train.values[:,-1]

Dividiremos nuestros datos de entrenamiento en un nuevo conjunto de entrenamiento y un conjunto de test, para así poder comparar el rendimiento y la precisión de los distintos algoritmos que utilicemos para predecir la actividad de los pacientes. Dedicaremos un 30% de los datos para realizar tests.

In [ ]:
from sklearn.model_selection import train_test_split
trainData, testData, trainLabels, testLabels = train_test_split(X,Y,test_size=0.3, random_state=4)

Veamos el tamaño de estos dos conjuntos de datos.

In [ ]:
(len(trainData),len(testData))

(8399, 3600)

En nuestro caso, es claro que debemos resolver un problema de clasificación múltiple, pues el modelo debe ser capaz de predecir cualquiera de las 6 actividades en base al resto de datos que poseemos de cada paciente.

Para ello, consideraremos los siguientes métodos: KNN, Árbol de decisión, Rgresión logística, SVM con kernel sigmoide, Perceptrón y Perceptrón multicapa. Cargaremos las librerías correspondientes para poder trabajar con estos métodos.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

Ahora, para inicializar cada clasificador debemos elegir el valor de sus respectivos hiperparámetros. Con el objetivo de hacer esta elección de forma óptima para que cada algoritmo produzca los mejores resultados posibles, combinaremos el k-fold cross validation con GridSearch. De este modo, ajustaremos de manera óptima los hiperparámetros de cada algoritmo.

Comencemos por el KNN. Primero, cargaremos la librería que utilizaremos para el GridSearch, que ya utiliza de por sí el cross validation.

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
modelKNN = KNeighborsClassifier()

El hiperparámetro que queremos seleccionar es n_neighbors. 

In [ ]:
tuned_parameters = [{'n_neighbors': range(1,26,2)}];
scores = ['precision', 'recall']

Definimos una función que, dado el modelo, los parámetros a optimizar, los objetivos y el número de folds del cross validation, busque el mejor conjunto de hiperparámetros.

In [ ]:
def gridsearch(model,parameters,scores,k):
    for score in scores:
        print("# Tuning hyper-parameters for %s" % score)
        print()

        clf = GridSearchCV(model, parameters, cv=k,
                           scoring='%s_macro' % score)
        clf.fit(trainData, trainLabels)

        print("Best parameters set found on development set:")
        print()
        print(clf.best_params_)
        print()
        print("Grid scores on development set:")
        print()
        means = clf.cv_results_['mean_test_score']
        stds = clf.cv_results_['std_test_score']
        for mean, std, params in zip(means, stds, clf.cv_results_['params']):
            print("%0.3f (+/-%0.03f) for %r"
                  % (mean, std * 2, params))
        print()

Aplicamos el GridSearch.

In [ ]:
gridsearch(modelKNN,tuned_parameters,scores,5)

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'n_neighbors': 5}

Grid scores on development set:

0.571 (+/-0.021) for {'n_neighbors': 1}
0.577 (+/-0.012) for {'n_neighbors': 3}
0.591 (+/-0.029) for {'n_neighbors': 5}
0.590 (+/-0.030) for {'n_neighbors': 7}
0.590 (+/-0.031) for {'n_neighbors': 9}
0.588 (+/-0.029) for {'n_neighbors': 11}
0.587 (+/-0.024) for {'n_neighbors': 13}
0.583 (+/-0.029) for {'n_neighbors': 15}
0.579 (+/-0.037) for {'n_neighbors': 17}
0.575 (+/-0.034) for {'n_neighbors': 19}
0.571 (+/-0.037) for {'n_neighbors': 21}
0.570 (+/-0.030) for {'n_neighbors': 23}
0.566 (+/-0.034) for {'n_neighbors': 25}

# Tuning hyper-parameters for recall

Best parameters set found on development set:

{'n_neighbors': 5}

Grid scores on development set:

0.570 (+/-0.022) for {'n_neighbors': 1}
0.580 (+/-0.017) for {'n_neighbors': 3}
0.587 (+/-0.032) for {'n_neighbors': 5}
0.585 (+/-0.036) for {'n_neighbors': 7}
0.585 (+/-0.031) for {'n_neighbo

Como vemos, la mejor opción es n_neighbors=5. Por tanto, volvemos a inicializar el clasificador con este hiperparámetro, y entrenamos el modelo.

In [ ]:
modelKNN = KNeighborsClassifier(n_neighbors=5)
modelKNN.fit(trainData,trainLabels)

KNeighborsClassifier()

Ahora, con el objetivo de comprobar lo acertadas que son las predicciones de este modelo, recurrimos al conjunto de test que hemos generado. Para ello, utilizaremos la función classification_report de la librería sklearn.

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print("EVALUACIÓN EN CONJUNTO DE TEST USANDO KNN")
predictionsKNN = modelKNN.predict(testData)
print(classification_report(testLabels, predictionsKNN))

EVALUACIÓN EN CONJUNTO DE TEST USANDO KNN
              precision    recall  f1-score   support

         0.0       0.73      0.73      0.73       997
         1.0       0.55      0.64      0.59        86
         2.0       0.58      0.65      0.61       581
         3.0       0.55      0.58      0.56       770
         4.0       0.55      0.55      0.55       778
         5.0       0.56      0.38      0.45       388

    accuracy                           0.61      3600
   macro avg       0.59      0.59      0.58      3600
weighted avg       0.61      0.61      0.60      3600



Como vemos, la precisión de este algoritmo al predecir cada actividad gira en torno al 60%.

Ahora, para hacernos una idea más clara de lo bien que ha predicho el modelo cada una de las actividades (y si confunde o no varias de ellas), recurriremos a la matriz de confusión.

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
print("MATRIZ DE CONFUSIÓN USANDO KNN")
print(confusion_matrix(testLabels, predictionsKNN))

MATRIZ DE CONFUSIÓN USANDO KNN
[[723  10  45  71 107  41]
 [  6  55  23   0   1   1]
 [ 45  17 377 106  28   8]
 [ 84   6 115 448 106  11]
 [ 82   6  67 138 430  55]
 [ 47   6  20  58 109 148]]


En la matriz de confusión vemos que predice bien a los pacientes que estaban de pie, pero da algunos problemas con el resto.

Repitamos este mismo proceso para el resto de clasificadores. Por ejemplo, continuamos con un árbol de decisión. En este caso no necesitamos seleccionar ningún hiperparámetro, por lo que directamente entrenaremos el modelo.

In [ ]:
modelArbol= DecisionTreeClassifier(random_state=4)
modelArbol.fit(trainData,trainLabels)

DecisionTreeClassifier(random_state=4)

Una vez entrenado, comprobamos el porcentaje de aciertos del árbol de decisión.

In [ ]:
print("EVALUACIÓN EN CONJUNTO DE TEST USANDO ÁRBOL DE DECISIÓN")
predictionsArbol = modelArbol.predict(testData)
print(classification_report(testLabels, predictionsArbol))

EVALUACIÓN EN CONJUNTO DE TEST USANDO ÁRBOL DE DECISIÓN
              precision    recall  f1-score   support

         0.0       0.93      0.80      0.86       997
         1.0       0.67      0.70      0.69        86
         2.0       0.66      0.68      0.67       581
         3.0       0.57      0.58      0.57       770
         4.0       0.55      0.60      0.57       778
         5.0       0.53      0.55      0.54       388

    accuracy                           0.66      3600
   macro avg       0.65      0.65      0.65      3600
weighted avg       0.68      0.66      0.67      3600



Con el árbol de decisión obtenemos una precisión en torno al 70%.

Veamos ahora la matriz de confusión.

In [ ]:
print("MATRIZ DE CONFUSIÓN USANDO ÁRBOL DE DECISIÓN")
print(confusion_matrix(testLabels, predictionsArbol))

MATRIZ DE CONFUSIÓN USANDO ÁRBOL DE DECISIÓN
[[802   2  23  41  69  60]
 [  0  60  24   0   2   0]
 [  5  20 396 116  37   7]
 [ 17   5 107 449 170  22]
 [ 21   1  33 155 465 103]
 [ 18   1  13  33 108 215]]


Vemos que de nuevo, los pacientes que estaban de pie se predicen muy bien, y el resto con algunos problemas.

Continuamos con la regresión logística.

In [ ]:
modelLR = LogisticRegression()

In [ ]:
tuned_parameters = [{'solver': ['newton-cg', 'lbfgs'], 'penalty': ['l2', 'none'],
                     'C': [0.01, 0.1,1, 10]},
                    {'solver': ['liblinear'], 'penalty': ['l2', 'l1'],
                     'C': [0.01, 0.1,1, 10]}]

In [ ]:
gridsearch(modelLR,tuned_parameters,scores,5)

# Tuning hyper-parameters for precision



/usr/local/lib/python3.7/dist-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/dist-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to c

Best parameters set found on development set:

{'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}

Grid scores on development set:

0.200 (+/-0.020) for {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
0.224 (+/-0.035) for {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
0.205 (+/-0.017) for {'C': 0.01, 'penalty': 'none', 'solver': 'newton-cg'}
0.226 (+/-0.089) for {'C': 0.01, 'penalty': 'none', 'solver': 'lbfgs'}
0.195 (+/-0.020) for {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.255 (+/-0.136) for {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.205 (+/-0.017) for {'C': 0.1, 'penalty': 'none', 'solver': 'newton-cg'}
0.226 (+/-0.089) for {'C': 0.1, 'penalty': 'none', 'solver': 'lbfgs'}
0.201 (+/-0.019) for {'C': 1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.219 (+/-0.075) for {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.205 (+/-0.017) for {'C': 1, 'penalty': 'none', 'solver': 'newton-cg'}
0.226 (+/-0.089) for {'C': 1, 'penalty': 'none', 'solver': 'lbfgs'}
0.199 (+/-0.024) for {'C

/usr/local/lib/python3.7/dist-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/dist-packages/scipy/optimize/linesearch.py:314: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/scipy/optimize/linesearch.py:466: LineSearchWarning: The line search algorithm did not converge
  warn('The line search algorithm did not converge', LineSearchWarning)
/usr/local/lib/python3.7/

Best parameters set found on development set:

{'C': 0.01, 'penalty': 'l1', 'solver': 'liblinear'}

Grid scores on development set:

0.203 (+/-0.009) for {'C': 0.01, 'penalty': 'l2', 'solver': 'newton-cg'}
0.190 (+/-0.009) for {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}
0.203 (+/-0.016) for {'C': 0.01, 'penalty': 'none', 'solver': 'newton-cg'}
0.191 (+/-0.006) for {'C': 0.01, 'penalty': 'none', 'solver': 'lbfgs'}
0.202 (+/-0.011) for {'C': 0.1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.191 (+/-0.006) for {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.203 (+/-0.016) for {'C': 0.1, 'penalty': 'none', 'solver': 'newton-cg'}
0.191 (+/-0.006) for {'C': 0.1, 'penalty': 'none', 'solver': 'lbfgs'}
0.204 (+/-0.009) for {'C': 1, 'penalty': 'l2', 'solver': 'newton-cg'}
0.189 (+/-0.012) for {'C': 1, 'penalty': 'l2', 'solver': 'lbfgs'}
0.203 (+/-0.016) for {'C': 1, 'penalty': 'none', 'solver': 'newton-cg'}
0.191 (+/-0.006) for {'C': 1, 'penalty': 'none', 'solver': 'lbfgs'}
0.203 (+/-0.016) fo

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


En este caso vemos que la elección de hiperparámetros óptima es C=0.1, penalty='l2' y solver='lbfgs'. Por tanto, inicializamos el clasificador y entrenamos el modelo.

In [ ]:
modelLR= LogisticRegression(C=0.1, penalty='l2', solver='lbfgs', random_state=4)
modelLR.fit(trainData,trainLabels)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(C=0.1, random_state=4)

Ahora, veamos el porcentaje de aciertos.

In [ ]:
print("EVALUACIÓN EN CONJUNTO DE TEST USANDO REGRESIÓN LOGÍSTICA")
predictionsLR = modelLR.predict(testData)
print(classification_report(testLabels, predictionsLR))

EVALUACIÓN EN CONJUNTO DE TEST USANDO REGRESIÓN LOGÍSTICA
              precision    recall  f1-score   support

         0.0       0.30      0.92      0.45       997
         1.0       0.00      0.00      0.00        86
         2.0       0.45      0.06      0.11       581
         3.0       0.26      0.09      0.14       770
         4.0       0.27      0.05      0.08       778
         5.0       0.00      0.00      0.00       388

    accuracy                           0.29      3600
   macro avg       0.21      0.19      0.13      3600
weighted avg       0.27      0.29      0.19      3600



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Como vemos, este modelo predice de forma muy errónea nuestros datos, pues ha tenido una precisión de en torno al 30%.

Veamos la matriz de confusión.

In [ ]:
print("MATRIZ DE CONFUSIÓN USANDO REGRESIÓN LOGÍSTICA")
print(confusion_matrix(testLabels, predictionsLR))

MATRIZ DE CONFUSIÓN USANDO REGRESIÓN LOGÍSTICA
[[915   0   4  60  18   0]
 [ 77   0   0   0   4   5]
 [467   0  35  34  45   0]
 [661   0  13  72  24   0]
 [641   0  10  88  38   1]
 [336   0  16  23  13   0]]


Atendiendo a la matriz de confusión, es claro que todas las actividades se predicen muy mal.

Continuamos con el SVM de kernel sigmoide.

In [ ]:
modelSVMSigmoide = SVC(kernel="sigmoid",gamma=2.0,coef0=1.0)
modelSVMSigmoide.fit(trainData,trainLabels)

SVC(coef0=1.0, gamma=2.0, kernel='sigmoid')

Una vez entrenado el modelo, comprobamos su porcentaje de acierto.

In [ ]:
print("EVALUACIÓN EN CONJUNTO DE TEST USANDO SVM (SIGMOIDE)")
predictionsSVMSigmoide = modelSVMSigmoide.predict(testData)
print(classification_report(testLabels, predictionsSVMSigmoide))

EVALUACIÓN EN CONJUNTO DE TEST USANDO SVM (SIGMOIDE)
              precision    recall  f1-score   support

         0.0       0.28      1.00      0.43       997
         1.0       0.00      0.00      0.00        86
         2.0       0.00      0.00      0.00       581
         3.0       0.00      0.00      0.00       770
         4.0       0.00      0.00      0.00       778
         5.0       0.00      0.00      0.00       388

    accuracy                           0.28      3600
   macro avg       0.05      0.17      0.07      3600
weighted avg       0.08      0.28      0.12      3600



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


De nuevo, vemos que la precisión obtenida es muy baja.

Veamos la matriz de confusión de este modelo.

In [ ]:
print("MATRIZ DE CONFUSIÓN USANDO SVM (SIGMOIDE)")
print(confusion_matrix(testLabels, predictionsSVMSigmoide))

MATRIZ DE CONFUSIÓN USANDO SVM (SIGMOIDE)
[[997   0   0   0   0   0]
 [ 86   0   0   0   0   0]
 [581   0   0   0   0   0]
 [770   0   0   0   0   0]
 [778   0   0   0   0   0]
 [388   0   0   0   0   0]]


Vemos que el modelo ha predicho que todos los pacientes estaban de pie, lo cual es claramente erróneo.

Seguimos con el Perceptrón.

In [ ]:
modelPerceptron= Perceptron(max_iter=10,eta0=1.0,random_state=4)
modelPerceptron.fit(trainData,trainLabels)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:700: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning,


Perceptron(max_iter=10, random_state=4)

Comprobamos su porcentaje de acierto.

In [ ]:
print("EVALUACIÓN EN CONJUNTO DE TEST USANDO PERCEPTRON")
predictionsPerceptron = modelPerceptron.predict(testData)
print(classification_report(testLabels, predictionsPerceptron))

EVALUACIÓN EN CONJUNTO DE TEST USANDO PERCEPTRON
              precision    recall  f1-score   support

         0.0       0.40      0.21      0.28       997
         1.0       0.00      0.00      0.00        86
         2.0       0.26      0.47      0.34       581
         3.0       0.00      0.00      0.00       770
         4.0       0.26      0.66      0.37       778
         5.0       0.00      0.00      0.00       388

    accuracy                           0.28      3600
   macro avg       0.15      0.22      0.16      3600
weighted avg       0.21      0.28      0.21      3600



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Calculamos su matriz de confusión.

In [ ]:
print("MATRIZ DE CONFUSIÓN USANDO PERCEPTRON")
print(confusion_matrix(testLabels, predictionsPerceptron))

MATRIZ DE CONFUSIÓN USANDO PERCEPTRON
[[208   0 256   0 533   0]
 [  5   0  61   0  20   0]
 [ 67   2 275   0 237   0]
 [130   2 198   0 440   0]
 [ 77   0 184   0 517   0]
 [ 28   0  81   0 279   0]]


Vemos que, de nuevo, el modelo tiene bastantes problemas con las predicciones.

Y por último, construimos el modelo del Perceptrón multicapa.

In [ ]:
modelMultiLayerPerceptron= MLPClassifier()

In [ ]:
tuned_parameters = [{'hidden_layer_sizes': [(5,2), (3,3,3), (5,3,2), (5,4,3,2)], 'activation': ['identity', 'logistic', 'tanh', 'relu'], 'learning_rate_init': [0.001, 0.01, 0.1],
                     'momentum': [0.9, 0.95, 0.99]}]

In [ ]:
gridsearch(modelMultiLayerPerceptron,tuned_parameters,scores,5)

# Tuning hyper-parameters for precision



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Pr

Best parameters set found on development set:

{'activation': 'identity', 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.001, 'momentum': 0.99}

Grid scores on development set:

0.128 (+/-0.066) for {'activation': 'identity', 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.001, 'momentum': 0.9}
0.133 (+/-0.081) for {'activation': 'identity', 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.001, 'momentum': 0.95}
0.196 (+/-0.074) for {'activation': 'identity', 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.001, 'momentum': 0.99}
0.112 (+/-0.101) for {'activation': 'identity', 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.01, 'momentum': 0.9}
0.191 (+/-0.081) for {'activation': 'identity', 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.01, 'momentum': 0.95}
0.170 (+/-0.122) for {'activation': 'identity', 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.01, 'momentum': 0.99}
0.125 (+/-0.113) for {'activation': 'identity', 'hidden_layer_sizes': 

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multi

Best parameters set found on development set:

{'activation': 'logistic', 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.001, 'momentum': 0.99}

Grid scores on development set:

0.165 (+/-0.028) for {'activation': 'identity', 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.001, 'momentum': 0.9}
0.163 (+/-0.042) for {'activation': 'identity', 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.001, 'momentum': 0.95}
0.181 (+/-0.033) for {'activation': 'identity', 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.001, 'momentum': 0.99}
0.181 (+/-0.013) for {'activation': 'identity', 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.01, 'momentum': 0.9}
0.181 (+/-0.017) for {'activation': 'identity', 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.01, 'momentum': 0.95}
0.172 (+/-0.038) for {'activation': 'identity', 'hidden_layer_sizes': (5, 2), 'learning_rate_init': 0.01, 'momentum': 0.99}
0.171 (+/-0.068) for {'activation': 'identity', 'hidden_layer_sizes': 

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Por tanto, la elección óptima de los hiperparámetros es hidden_layer_sizes=(5,2), activation='identity', learning_rate_init=0.001, y momentum=0.99. Inicialicemos de nuevo el clasificador con estos hiperparámetros y entrenemos el modelo.

In [ ]:
modelMultiLayerPerceptron= MLPClassifier(hidden_layer_sizes=(5,2),activation='identity',learning_rate_init=0.001,momentum=0.99)
modelMultiLayerPerceptron.fit(trainData,trainLabels)

MLPClassifier(activation='identity', hidden_layer_sizes=(5, 2), momentum=0.99)

Comprobamos su porcentaje de aciertos.

In [ ]:
print("EVALUACIÓN EN CONJUNTO DE TEST USANDO PERCEPTRON MULTICAPA")
predictionsMultiLayerPerceptron = modelMultiLayerPerceptron.predict(testData)
print(classification_report(testLabels, predictionsMultiLayerPerceptron))

EVALUACIÓN EN CONJUNTO DE TEST USANDO PERCEPTRON MULTICAPA
              precision    recall  f1-score   support

         0.0       0.23      0.55      0.33       997
         1.0       0.03      0.06      0.04        86
         2.0       0.46      0.06      0.11       581
         3.0       0.00      0.00      0.00       770
         4.0       0.00      0.00      0.00       778
         5.0       0.07      0.17      0.10       388

    accuracy                           0.18      3600
   macro avg       0.13      0.14      0.09      3600
weighted avg       0.15      0.18      0.12      3600



Y finalmente calculamos su matriz de confusión.

In [ ]:
print("MATRIZ DE CONFUSIÓN USANDO PERCEPTRON MULTICAPA")
print(confusion_matrix(testLabels, predictionsMultiLayerPerceptron))

MATRIZ DE CONFUSIÓN USANDO PERCEPTRON MULTICAPA
[[551  51   2   2   5 386]
 [ 81   5   0   0   0   0]
 [397  29  35   0   0 120]
 [480  42  13   0   0 235]
 [571  42  10   1   0 154]
 [296  10  16   0   0  66]]


Como vemos, el modelo con mayor precisión en sus predicciones sobre los datos de test es el que utiliza el árbol de decisión, y por tanto será este el que utilizaremos para predecir nuestros datos.

Equivalentemente al estudio realizado, también podríamos realizar un estudio estadístico para comparar los distintos métodos y elegir el que mejores predicciones dé acerca de la actividad de los pacientes. Para ello, deberíamos cargar la siguiente librería:

In [ ]:
!pip install StatisticalAnalysis

  Created wheel for StatisticalAnalysis: filename=StatisticalAnalysis-0.0.5-py2.py3-none-any.whl size=13122 sha256=5009883dad27ee9f174eccce56c7af4b771c6bc4ad4ceb898430055da22ddba8
  Stored in directory: /root/.cache/pip/wheels/37/d7/b4/16b0207c2fe82f25f64c17e61c8c2027aea5fad65d059c2394
Successfully built StatisticalAnalysis


In [ ]:
from StatisticalAnalysis import compare_methods
from scipy.stats import randint as sp_randint

Después, introducimos los datos de todos los algoritmos que queremos considerar.

In [ ]:
# Árbol de decisión
clfTree = DecisionTreeClassifier(random_state=4)
param_distTree = {"min_samples_split": sp_randint(3, 30)}
# SVM
clfSVC = SVC(random_state=84)
param_distSVC = {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],'kernel': ['rbf'], 'class_weight':['balanced', None]}
# KNN
clfKNN = KNeighborsClassifier()
param_distKNN = {'n_neighbors':sp_randint(3, 30)}
# Regresión logística
clfLR = LogisticRegression(random_state=84)
param_distLR = {'C': [0.1,0.5,1, 10, 100, 1000]}
# Red neuronal
clfMLP = MLPClassifier(random_state=84)
param_distMLP = {'activation': ['identity', 'logistic', 'tanh', 'relu'], 'solver': ['lbfgs', 'sgd', 'adam'],
                 'alpha': sp_randint(0.0001, 1), 'learning_rate': ['constant', 'invscaling', 'adaptive'],
                 'hidden_layer_sizes': [(5,2), (3,3,3), (5,3,2), (5,4,3,2)],
                 'momentum': [0.9, 0.95, 0.99]}

In [ ]:
listAlgorithms = [clfTree,clfSVC,clfKNN,clfLR,clfMLP]
listParams = [param_distTree,param_distSVC,param_distKNN,param_distLR,param_distMLP]
listNames = ["Arbol", "SVM", "KNN", "LR", "MLP"]

Y finalmente comparamos los métodos. Es importante notar que este proceso puede ser muy costoso computacionalmente, en función del tamaño de nuestro dataset, de los algoritmos considerados y de los hiperparámetros introducidos que queramos optimizar.

In [ ]:
compare_methods(X,Y,listAlgorithms,listParams,listNames,metric='accuracy')

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 6 is smaller than n_iter=20. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 16 is smaller than n_iter=20. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/loca

In the multiclass problem, only accuracy can be used as metric


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In the multiclass problem, only accuracy can be used as metric


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 6 is smaller than n_iter=20. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the dat

In the multiclass problem, only accuracy can be used as metric


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In the multiclass problem, only accuracy can be used as metric
In the multiclass problem, only accuracy can be used as metric


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 16 is smaller than n_iter=20. Running 16 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_

En nuestro caso, nos quedaremos con el árbol de decisión, ya que claramente ha sido el algoritmo que mejores predicciones nos ha aportado en el anterior estudio. Como se puede observar, la última celda no ha terminado de ejecutarse del todo; se ha tenido que interrumpir su ejecución debido a que el método no terminó tras más de 4 horas de cómputo. En cualquier caso, es previsible que este estudio estadístico corrobore los resultados obtenidos al comparar los métodos manualmente; es decir, es previsible que el estudio estadístico determine el árbol de decisión como la mejor opción.

En el siguiente notebook aplicaremos las técnicas de preprocesado de los datos ya expuestas, y daremos el modelo que finalmente sea capaz de realizar predicciones fiables sobre nuestro conjunto de test.